# Explore Personalization Using Multi-Dataset

#### In this notebook, we explore the impact of using three datasets (M2sleep, HeartS, BiheartS) in model personlization. The three datasets are collected in the wild, through different longitudinal data collection campaigns.

#### - The three datasets use Empatica E4 wristband device, to collect several physiological signals (BVP, EDA, TEMP)

#### - In this notebook, we are going to explore the impact of dataset bias on the personlization sleep quality recognition task 

#### - To this goal, we are going to compare three machine learning model (population model, one-user-only model, and personalized model) using 

##### ***1. Single dataset***

##### ***2. Mutiple datasets***

##### ***2. Mixed datasets***

#### In this notebook, we implement the ***Single dataset*** scenario with the prequential and the holdout evaluation methods


### 0. Load packages

In [ ]:
from sys import path 
path.append("../")
from source.models.prequential_online_learning import PrequentialOnlineLearning as PrequentialOnlineLearning
from source.utils.helper_functions import visualize_models_results
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt


warnings.filterwarnings("ignore")

# Generate a list of 50 unique integers to use as seeds
seed_list = np.arange(50)  # Create an array with integers from 0 to 49
# Convert to list
seed_list = seed_list.tolist()


